In [4]:
import torch
from collections import defaultdict
import numpy as np
import mcubes
import trimesh

from models.rendering import *
from models.nerf import *

from datasets import dataset_dict

from utils import load_ckpt

# Load model and data

In [3]:
import os
!curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
!tar xzf 1.10.0.tar.gz
os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
!pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   118  100   118    0     0    634      0 --:--:-- --:--:-- --:--:--   634
100  404k    0  404k    0     0   390k      0 --:--:--  0:00:01 --:--:--  543k
  Cloning https://github.com/facebookresearch/pytorch3d.git (to revision stable) to /tmp/pip-req-build-kv2nqwr2
  Running command git clone -q https://github.com/facebookresearch/pytorch3d.git /tmp/pip-req-build-kv2nqwr2
  Running command git checkout -q 3c15a6c2469249c8b90a4f3e41e34350b8051b92


In [1]:
from lietorch import SO3, SE3, Sim3

In [2]:
!cat /usr/local/cuda/version.txt

CUDA Version 10.2.89


In [6]:
a = torch.tensor([[0., 0., 1., 0., 0.,0.]], device='cuda',requires_grad=True) # [t,r] (1,6)
b= torch.eye(4,4)
print(a)
print("posse:{}".format(SE3.exp(a).matrix()))

tensor([[0., 0., 1., 0., 0., 0.]], device='cuda:0', requires_grad=True)
posse:tensor([[[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 1.],
         [0., 0., 0., 1.]]], device='cuda:0', grad_fn=<ViewBackward>)


In [ ]:
# Change here #
img_wh = (800, 800) # full resolution of the input images
dataset_name = 'blender' # blender or llff (own data)
scene_name = 'lego' # whatever you want
root_dir = '/home/ubuntu/data/nerf_example_data/nerf_synthetic/lego/' # the folder containing data
ckpt_path = 'ckpts/exp2/epoch=05.ckpt' # the model path
###############

kwargs = {'root_dir': root_dir,
          'img_wh': img_wh}
if dataset_name == 'llff':
    kwargs['spheric_poses'] = True
    kwargs['split'] = 'test'
else:
    kwargs['split'] = 'train'
    
chunk = 1024*32
dataset = dataset_dict[dataset_name](**kwargs)

embedding_xyz = Embedding(3, 10)
embedding_dir = Embedding(3, 4)

nerf_fine = NeRF()
load_ckpt(nerf_fine, ckpt_path, model_name='nerf_fine')
nerf_fine.cuda().eval();

# Search for tight bounds of the object (trial and error!)

In [ ]:
### Tune these parameters until the whole object lies tightly in range with little noise ###
N = 128 # controls the resolution, set this number small here because we're only finding
        # good ranges here, not yet for mesh reconstruction; we can set this number high
        # when it comes to final reconstruction.
xmin, xmax = -1.2, 1.2 # left/right range
ymin, ymax = -1.2, 1.2 # forward/backward range
zmin, zmax = -1.2, 1.2 # up/down range
## Attention! the ranges MUST have the same length!
sigma_threshold = 50. # controls the noise (lower=maybe more noise; higher=some mesh might be missing)
############################################################################################

x = np.linspace(xmin, xmax, N)
y = np.linspace(ymin, ymax, N)
z = np.linspace(zmin, zmax, N)

xyz_ = torch.FloatTensor(np.stack(np.meshgrid(x, y, z), -1).reshape(-1, 3)).cuda()
dir_ = torch.zeros_like(xyz_).cuda()

with torch.no_grad():
    B = xyz_.shape[0]
    out_chunks = []
    for i in range(0, B, chunk):
        xyz_embedded = embedding_xyz(xyz_[i:i+chunk]) # (N, embed_xyz_channels)
        dir_embedded = embedding_dir(dir_[i:i+chunk]) # (N, embed_dir_channels)
        xyzdir_embedded = torch.cat([xyz_embedded, dir_embedded], 1)
        out_chunks += [nerf_fine(xyzdir_embedded)]
    rgbsigma = torch.cat(out_chunks, 0)
    
sigma = rgbsigma[:, -1].cpu().numpy()
sigma = np.maximum(sigma, 0)
sigma = sigma.reshape(N, N, N)

# The below lines are for visualization, COMMENT OUT once you find the best range and increase N!
vertices, triangles = mcubes.marching_cubes(sigma, sigma_threshold)
mesh = trimesh.Trimesh(vertices/N, triangles)
mesh.show()

In [ ]:
# # You can already export "colorless" mesh if you don't need color
# mcubes.export_mesh(vertices, triangles, f"{scene_name}.dae")

# Generate .vol file for volume rendering in Unity

In [ ]:
assert N==512, \
    'Please set N to 512 in the two above cell! Remember to comment out the visualization code (last 3 lines)!'

a = 1-np.exp(-(xmax-xmin)/N*sigma)
a = a.flatten()
rgb = (rgbsigma[:, :3].numpy()*255).astype(np.uint32)
i = np.where(a>0)[0] # valid indices (alpha>0)

rgb = rgb[i]
a = a[i]
s = rgb.dot(np.array([1<<24, 1<<16, 1<<8])) + (a*255).astype(np.uint32)
res = np.stack([i, s], -1).astype(np.uint32).flatten()
with open(f'{scene_name}.vol', 'wb') as f:
    f.write(res.tobytes())

# Extract colored mesh

Once you find the best range, now **RESTART** the notebook, and copy the configs to the following cell
and execute it.

In [ ]:
# Copy the variables you have above here! ####
img_wh = (800, 800) # full resolution of the input images
dataset_name = 'blender' # blender or llff (own data)
scene_name = 'lego' # whatever you want
root_dir = '/home/ubuntu/data/nerf_example_data/nerf_synthetic/lego/' # the folder containing data
ckpt_path = 'ckpts/exp2/epoch=05.ckpt' # the model path

N = 128 # controls the resolution, set this number small here because we're only finding
        # good ranges here, not yet for mesh reconstruction; we can set this number high
        # when it comes to final reconstruction.
xmin, xmax = -1.2, 1.2 # left/right range
ymin, ymax = -1.2, 1.2 # forward/backward range
zmin, zmax = -1.2, 1.2 # up/down range
sigma_threshold = 50. # controls the noise (lower=maybe more noise; higher=some mesh might be missing)
###############################################

import os
os.environ['ROOT_DIR'] = root_dir
os.environ['DATASET_NAME'] = dataset_name
os.environ['SCENE_NAME'] = scene_name
os.environ['IMG_SIZE'] = f"{img_wh[0]} {img_wh[1]}"
os.environ['CKPT_PATH'] = ckpt_path
os.environ['N_GRID'] = "256" # final resolution. You can set this number high to preserve more details
os.environ['X_RANGE'] = f"{xmin} {xmax}"
os.environ['Y_RANGE'] = f"{ymin} {ymax}"
os.environ['Z_RANGE'] = f"{zmin} {zmax}"
os.environ['SIGMA_THRESHOLD'] = str(sigma_threshold)
os.environ['OCC_THRESHOLD'] = "0.2" # probably doesn't require tuning. If you find the color is not close
                                    # to real, try to set this number smaller (the effect of this number
                                    # is explained in my youtube video)

!python extract_color_mesh.py \
    --root_dir $ROOT_DIR \
    --dataset_name $DATASET_NAME \
    --scene_name $SCENE_NAME \
    --img_wh $IMG_SIZE \
    --ckpt_path $CKPT_PATH \
    --N_grid $N_GRID \
    --x_range $X_RANGE \
    --y_range $Y_RANGE \
    --z_range $Z_RANGE \
    --sigma_threshold $SIGMA_THRESHOLD \
    --occ_threshold $OCC_THRESHOLD